In [939]:
from bisect import bisect_left
from configparser import ConfigParser
from sys import argv

from lxml import etree
from shapely.affinity import rotate
from shapely.geometry import Polygon

from constants import (BLOCK_REGISTRY,
                       MULTIPLIER,
                       GROUND_HEIGHT,
                       BLOCK_STRING,
                       PIG_STRING,
                       LEVEL_TEMPLATE)
import random
import numpy as np
import obj_xml_generator as oxg


In [1006]:
class OBJmatrix:
    def __init__(self,svg_file_path,num_row,num_col):
        self.num_block_row = num_row
        self.num_block_column = num_col
        
        self.file_path = svg_file_path
        config = ConfigParser()
        config.read('../../config.ini')
        self.level_path = config.get('DEFAULT', 'LevelPath') + svg_file_name.split('/')[-1].split('.')[0] + '.xml'
        self.shape = self.__get_polygon_from_svg()
        self.matrix = self.__build_matrix()
        
        self.addition = lambda i: 1 if i%2==1 else 0
        self.set_all_blocks()
        self.get_outline()
        self.set_all_blocks()
        self.replace_to_rectangle()
        self.add_support()
        self.support_points.reverse()
        

        
        #generator levels
        self.generator_levels()
        
    def __get_polygon_from_svg(self):
    # Need to turn image upside down. The reason is that Potrace generates an
    # upside down SVG image and renders it correctly using "scale" function of
    # the "transform" SVG attribute. Since using the "scale" function is not
    # possible in Shapely, we just generate the shape as upside down and then
    # rotate it.
        return rotate(Polygon([tuple([float(c) for c in pair.split(',')])
                        for
                        pair
                        in
                        etree.parse(self.file_path).find('.//{http://www.w3.org/2000/svg}polygon').get('points').split()]),
                      180)

    def __build_matrix(self):
        #caculate the overall area and build simulation matrix
        minx,miny,maxx,maxy = self.shape.bounds
        width = (maxx-minx)/self.num_block_row 
        height = (maxy-miny)/self.num_block_column
        matrix = []
        for i in range(self.num_block_column):
            tam = []
            for j in range(self.num_block_row):
                poly = Polygon([(minx + j * width, miny + i * height),
                        (minx + j * width + width, miny + i * height),
                        (minx + j * width + width, miny + i * height + height),
                        (minx + j * width, miny + i * height + height)])
                if poly.intersection(self.shape).area > poly.area*0.1:
                    tam.append(1)
                else:
                    tam.append(0)
            matrix.append(tam)
        
        #extend the matrix size to do futher operations
        for i in matrix:
            for index in range(4):
                i.insert(0,0)
                i.append(0)
        for i in range(4):
            matrix.insert(0,[0 for i in range(len(matrix[0]))])
        return matrix
    
    def set_all_blocks(self):
        #calling set block type
        for i in range(len(self.matrix)-1):
            for j in range(1,len(self.matrix[0])-1):
                if self.matrix[i][j] != 0:
                    self.matrix[i][j] = self.set_block_type(i,j)

    def set_block_type(self,i,j):
    #     the method will convert the blocks into 4 basic block types 
    #     and return a integer which descripts the information about surrounding blocks of this blocks,
    #     1: this block have both downleft and downright blocks support it
    #     2: this block only have downleft block
    #     3: this block only have downright block
    #     4: this block nether have downleft or downright
        addition = self.addition(i)


        topleft,topright,downleft,downright = [
                                               self.matrix[i-1][j-1+addition] != 0 if i != 0 else False,
                                               self.matrix[i-1][j+addition] != 0 if i !=0 else False,
                                               self.matrix[i+1][j-1+addition] != 0, 
                                               self.matrix[i+1][j+addition] != 0
                                              ]
#             At this time, if block dont have down left, label this block as 2, 
#             else if the block dont have down right, label as 4
#             else if the block neither have down left or down right, label as 3
        if downleft and downright:
            return 1
        elif not downleft and not downright:
            return 3
#                 if not topleft:
#                     self.matrix[i][j] = 0
#                     self.set_block_type(i-1,j+addition)
#                 if not topright:
#                     self.matrix[i][j] = 0
#                     self.set_block_type(i-1,j-1+addition)
        elif downleft:
#                 if not topleft:
#                     self.matrix[i][j] = 0
#                     self.set_block_type(i-1,j+addition)
#                 else:
#                     self.matrix[i][j] = 2
            return  2
        elif downright:
#                 if not topright:
#                     self.matrix[i][j] = 0
#                     self.set_block_type(i-1,j-1+addition)
#                 else:
#                     self.matrix[i][j] = 4
            return 4
                    
    def get_outline(self):
        #find the outline for each row in the matrix
        widths = []
        for i in range(len(self.matrix)):
            width = []
            copy = self.matrix[i].copy()
            for j in range(1,len(self.matrix[i])-1):
                bool_row = copy[j] != 0
                bool_plus = copy[j+1] != 0
                bool_minus = copy[j-1] != 0
#                 if bool_plus and bool_minus:   
                self.matrix[i][j] = 0
                if not bool_row and bool_plus:
                    left = j+1
                if not bool_plus and bool_row:
                    width.append([left,j])
            widths.append(width)
        self.widths = widths
    
    def replace_to_rectangle(self):
        #insert rectangle into the matrix
        self.support_points = []
        for i in range(len(self.widths)-1,-1,-1):
            if self.widths[i] != [] and (len(self.matrix)-1-i)%4 == 0:
                for left,right in self.widths[i]:
                    support_points = []
                    rectangles = []
                    width = right-left+1
                    while(width > 0):
                        if width >= 9:
                            rectangles.append([35,9])
                            width -= 9
                        elif width >= 7:
                            rectangles.append([34,7])
                            width -= 7
                        else:
                            rectangles.append([33,4])
                            width -= 4

                    start = left + int(np.floor(width/2))
                    for rect in rectangles:
#                         print(str(i)+" width:"+str(width)+" "+str(rect)+"    "+str(start)+"  "+str(start+rect[1])+" "+str(start+int(rect[1]/2)))
                        support_points.append([start,start+rect[1]-1])
                        for index in range(start,start+rect[1]):                # clear the blocks
                            self.matrix[i][index] = 9
                        self.matrix[i][start+int(rect[1]/2)] = rect[0]
                        start += rect[1]
                    self.support_points.append([i,support_points])
                
    def add_support(self):
        #adding support blocks
        for row,rects in self.support_points:
            if row + 4 < len(self.matrix):
                for rect in rects:
                    for col in rect:
                        supported = False

                        if self.matrix[row+4][col] != 0:
                            for index in range(1,4):
                                if index%2 == 1:
                                    self.matrix[row+index][col+1+self.addition(row+index)] = 1
                                self.matrix[row+index][col] = 1
                            continue
                        for j in range(1,3):
                            if self.matrix[row+4][col+j] != 0:
                                x,y = (row,col)
                                self.matrix[x+1][y+1+self.addition(x)] = 4
                                for count in range(1,4):
                                    y += self.addition(x)
                                    x += 1
                                    self.matrix[x][y] = 4
                                supported = True
                                break
                            if self.matrix[row+4][col-j] != 0:
                                x,y = (row,col)
                                self.matrix[x+1][y-2+self.addition(x)] = 4
                                for count in range(1,4):
                                    y += -1 + self.addition(x)
                                    x += 1
                                    self.matrix[x][y] = 2
                                supported = True
                                break
                        if not supported:
                            self.matrix[row+2][col] = 6

    
    def generator_levels(self):
        #length_square = 0.24055
        #height_square = 0.225
        length = 0.24055
        height = 0.225

        length_platform = 0.24055
        height_platform = 0.225

        #block = [[ random.randint(0,1) for j in range(5)] for i in range(75)]  #randomly create matrix
        block = self.matrix
        ftblock = np.fliplr(np.transpose(block))
        primary_block_elements = ''
        support_block_elements = ''
        
        for i in range(len(ftblock)):
            for j in range(len(ftblock[i])):
                if ftblock[i][j] == 32:
                    support_block_elements += oxg.get_object_string('Block',
                                                                    'RectTiny',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.4,
                                                                    block_material='stone',
                                                                    spining = 0)
                elif ftblock[i][j] == 33:
                    support_block_elements += oxg.get_object_string('Block',
                                                                    'RectSmall',
                                                                    i * length + (0.12 if j%2==0 else 0)- 0.14, 
                                                                    height * j - 3.4,
                                                                    block_material='stone',
                                                                    spining = 0)
                elif ftblock[i][j] == 34:
                    support_block_elements += oxg.get_object_string('Block',
                                                                    'RectMedium',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.4,
                                                                    block_material='stone',
                                                                    spining = 0)
                elif ftblock[i][j] == 35:
                    support_block_elements += oxg.get_object_string('Block',
                                                                    'RectBig',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.4,
                                                                    block_material='stone',
                                                                    spining = 0)
                elif ftblock[i][j] == 5:
                    support_block_elements += oxg.get_object_string('Block',
                                                                    'TriangleHole',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.4,
                                                                    block_material='stone',
                                                                    spining = 180)
                elif ftblock[i][j] == 6:
                    support_block_elements += oxg.get_object_string('Platform',
                                                                    'Platform',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.4,
                                                                    block_material='',
                                                                    spining = 0)
                elif ftblock[i][j] == 7:
                    support_block_elements += oxg.get_object_string('Pig',
                                                                    'BasicSmall',
                                                                    i * length + (0.12 if j%2==0 else 0), 
                                                                    height * j - 3.415,
                                                                    block_material='',
                                                                    spining = -1)
                elif ftblock[i][j] != 0 and ftblock[i][j] != 9:
                    primary_block_elements += oxg.get_object_string('Block',
                                                                    'SquareTiny',
                                                                    i * length + (0.12 if j%2==0 else 0),
                                                                    height * j - 3.4,
                                                                    block_material='ice',
                                                                    spining = 0)
        xml = primary_block_elements + support_block_elements
        with open(self.level_path, 'w') as level_file:
                    level_file.write(LEVEL_TEMPLATE.strip().format(xml))
      
                    
                    
                    
    def __repr__(self):
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[i])):
                string = ''
                for i in range(0,len(self.matrix)):
                    if (len(self.matrix)-i-1)%2 == 0:
                        string += ' '
                    for j in self.matrix[i]:
                        string += str(j)[-1] + ' '
                    string +=  ('  ' if (len(self.matrix)-i-1)%2 else ' ') + str(i) + '\n'
                return string

In [1013]:
if __name__ == "__main__":
    svg_file_name = '../../Images/Edited/Animals/Cat-black-and-white-denoised-polygon.svg'
    obj = OBJmatrix(svg_file_name,30,60)
    obj.generator_levels()

In [1014]:
obj.support_points

[[7, [[30, 33]]],
 [7, [[5, 8]]],
 [11, [[27, 30], [31, 34]]],
 [11, [[4, 10]]],
 [15, [[25, 33]]],
 [15, [[16, 19]]],
 [15, [[3, 11], [12, 15]]],
 [19, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [23, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [27, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [31, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [35, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [39, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [43, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [47, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [51, [[3, 11], [12, 20], [21, 29], [30, 33]]],
 [55, [[7, 15], [16, 24], [25, 28]]],
 [59, [[11, 19], [20, 23]]],
 [63, [[14, 17], [18, 21]]]]

In [1015]:
count = 10

for i, rects in obj.support_points:
    for left,right in rects:
        j = int((left+right)/2)
        placeable = True
        for row in range(1,3):
            for col in range(-3,3):
                if obj.matrix[i-row][j+col] != 0:
                    placeable = False
        if placeable:
            obj.matrix[i-1][j+1] = 7
            count-=1
            print(i-1,j)
    if count <=0:
        break
obj.generator_levels()

6 31
6 6
14 17
62 15
